In [ ]:
!pip install tensorflow

In [1]:

import pandas as pd
import numpy as np

# There are eight datasets in total. They can be found here: https://www.kaggle.com/datasets/cicdataset/cicids2017

import warnings
warnings.filterwarnings("ignore")

In [2]:
df1=pd.read_csv("Wednesday-workingHours.pcap_ISCX.csv")
df2=pd.read_csv("Tuesday-WorkingHours.pcap_ISCX.csv")
df3=pd.read_csv("Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df4=pd.read_csv("Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df5=pd.read_csv("Monday-WorkingHours.pcap_ISCX.csv")
df6=pd.read_csv("Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df7=pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df8=pd.read_csv("Friday-WorkingHours-Morning.pcap_ISCX.csv")
df = pd.concat([df1,df2, df3, df4, df5, df6, df7, df8])
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
print(df.shape)

(2830743, 79)


In [3]:
print(df.shape)

(2830743, 79)


In [4]:
# Drop columns that are completely homogenous

df_nums = df.select_dtypes(include=[np.number])
print("The columns that are completely homogenous are")
for col in df_nums.columns:
    if (df_nums[col].max()-df_nums[col].min() == 0):
        print(col,)
df.drop([col for col in df_nums.columns if df_nums[col].max()-df_nums[col].min() == 0], axis=1, inplace=True)
print(df.shape)

The columns that are completely homogenous are
 Bwd PSH Flags
 Bwd URG Flags
Fwd Avg Bytes/Bulk
 Fwd Avg Packets/Bulk
 Fwd Avg Bulk Rate
 Bwd Avg Bytes/Bulk
 Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate
(2830743, 71)


In [5]:
# Drop NA rows

df.dropna(axis=0, inplace=True)
print(df.shape)

(2829385, 71)


In [6]:
# Normalization
df_nums = df.select_dtypes(include=[np.number])
df_nums = (df_nums-df_nums.min())/(df_nums.max()-df_nums.min())

for col in df_nums.columns:
    df[col] = df_nums[col]

df.dropna(axis=0, inplace=True)
for c in df.columns:
    print(c + " :", df[c].isna().sum())

 Destination Port : 0
 Flow Duration : 0
 Total Fwd Packets : 0
 Total Backward Packets : 0
Total Length of Fwd Packets : 0
 Total Length of Bwd Packets : 0
 Fwd Packet Length Max : 0
 Fwd Packet Length Min : 0
 Fwd Packet Length Mean : 0
 Fwd Packet Length Std : 0
Bwd Packet Length Max : 0
 Bwd Packet Length Min : 0
 Bwd Packet Length Mean : 0
 Bwd Packet Length Std : 0
Flow Bytes/s : 0
 Flow Packets/s : 0
 Flow IAT Mean : 0
 Flow IAT Std : 0
 Flow IAT Max : 0
 Flow IAT Min : 0
Fwd IAT Total : 0
 Fwd IAT Mean : 0
 Fwd IAT Std : 0
 Fwd IAT Max : 0
 Fwd IAT Min : 0
Bwd IAT Total : 0
 Bwd IAT Mean : 0
 Bwd IAT Std : 0
 Bwd IAT Max : 0
 Bwd IAT Min : 0
Fwd PSH Flags : 0
 Fwd URG Flags : 0
 Fwd Header Length : 0
 Bwd Header Length : 0
Fwd Packets/s : 0
 Bwd Packets/s : 0
 Min Packet Length : 0
 Max Packet Length : 0
 Packet Length Mean : 0
 Packet Length Std : 0
 Packet Length Variance : 0
FIN Flag Count : 0
 SYN Flag Count : 0
 RST Flag Count : 0
 PSH Flag Count : 0
 ACK Flag Count : 0
 U

In [7]:
df.shape

(2827876, 71)

In [11]:
label_counts = df[' Label'].value_counts()
print("Distinct label names and their counts:")
print(label_counts)

Distinct label names and their counts:
BENIGN                        2271320
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name:  Label, dtype: int64


In [12]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Apply label encoding to categorical columns
for col in [' Label']:
    df[col] = label_encoder.fit_transform(df[col])

# Display the first few rows of the encoded DataFrame
print(df.head())

    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.001221        0.000319            0.000000   
1           0.005936        0.000004            0.000046   
2           0.001343        0.000009            0.000041   
3           0.005936        0.000127            0.000073   
4           0.001343        0.000009            0.000036   

    Total Backward Packets  Total Length of Fwd Packets  \
0                 0.000003                 4.651163e-07   
1                 0.000017                 1.333333e-05   
2                 0.000021                 2.441860e-04   
3                 0.000041                 2.675969e-04   
4                 0.000021                 2.441860e-04   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                  9.153974e-09                0.000242   
1                  4.973659e-07                0.003183   
2                  4.805836e-06                0.063457   
3                  1.016091e-05                0

In [13]:
label_counts = df[' Label'].value_counts()
print("Distinct label names and their counts:")
print(label_counts)

Distinct label names and their counts:
0     2271320
4      230124
10     158804
2      128025
3       10293
7        7935
11       5897
6        5796
5        5499
1        1956
12       1507
14        652
9          36
13         21
8          11
Name:  Label, dtype: int64


In [14]:
# Selecting rows with specified labels
selected_labels = [9,13,8]

# Check the precision of the labels in the DataFrame

selected_rows = df[df[' Label'].isin(selected_labels)]

In [15]:
selected_rows.shape

(68, 71)

In [16]:
X = selected_rows.iloc[:, :-1].values  # Features (excluding the label column)
y = selected_rows.iloc[:, -1].values   # Labels

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split

In [42]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np

# Define WGAN-GP components
def make_generator_model(input_dim, output_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, use_bias=False, input_shape=(input_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(output_dim, activation='tanh'))
    return model

def make_discriminator_model(input_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(input_dim,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1))
    return model

def discriminator_loss(real_output, fake_output):
    real_loss = tf.reduce_mean(real_output)
    fake_loss = tf.reduce_mean(fake_output)
    return fake_loss - real_loss

def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

def gradient_penalty(discriminator, real_data, fake_data):
    batch_size = real_data.shape[0]
    alpha = tf.random.uniform([batch_size, 1], 0.0, 1.0)
    real_data = tf.cast(real_data, tf.float32)
    fake_data = tf.cast(fake_data, tf.float32)
    diff = fake_data - real_data
    interpolated = real_data + alpha * diff
    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        pred = discriminator(interpolated)
    grads = gp_tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=1))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

@tf.function
def train_step(real_data, generator, discriminator, generator_optimizer, discriminator_optimizer, batch_size, z_dim, gp_weight):
    noise = tf.random.normal([batch_size, z_dim])
    real_data = tf.cast(real_data, tf.float32)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_data = generator(noise, training=True)
        real_output = discriminator(real_data, training=True)
        fake_output = discriminator(fake_data, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        gp = gradient_penalty(discriminator, real_data, fake_data)
        total_disc_loss = disc_loss + gp * gp_weight

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(total_disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# WGAN-GP training function
def train_wgan_gp(X, epochs, batch_size, z_dim, gp_weight, generator, discriminator, generator_optimizer, discriminator_optimizer):
    dataset = tf.data.Dataset.from_tensor_slices(tf.cast(X, tf.float32)).shuffle(buffer_size=1024).batch(batch_size)

    for epoch in range(epochs):
        for real_data in dataset:
            if real_data.shape[0] != batch_size:
                continue  # Skip the batch if it does not match the required batch size
            train_step(real_data, generator, discriminator, generator_optimizer, discriminator_optimizer, batch_size, z_dim, gp_weight)

    noise = tf.random.normal([X.shape[0], z_dim])
    synthetic_data = generator(noise, training=False)
    return synthetic_data

# Set WGAN-GP parameters
z_dim = 10
gp_weight = 10.0
batch_size = 1
epochs = 10  # Adjust as needed

# Create WGAN-GP models and optimizers
generator = make_generator_model(z_dim, X.shape[1])
discriminator = make_discriminator_model(X.shape[1])
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Train WGAN-GP and generate synthetic data
synthetic_X = train_wgan_gp(X, epochs, batch_size, z_dim, gp_weight, generator, discriminator, generator_optimizer, discriminator_optimizer)
synthetic_y = np.random.choice(y, size=synthetic_X.shape[0])  # Assuming same distribution for labels

# Combine real and synthetic data
X_combined = np.vstack((X, synthetic_X))
y_combined = np.hstack((y, synthetic_y))


In [43]:
X_combined.shape

(8704, 70)

In [44]:
# Combine into a single DataFrame
df_combined = pd.DataFrame(X_combined)
df_combined['Label'] = y_combined

In [45]:
label_counts = df_combined['Label'].value_counts()
print("Distinct label names and their counts:")
print(label_counts)

Distinct label names and their counts:
9     4327
13    2586
8     1791
Name: Label, dtype: int64


In [46]:
df_combined.columns=df.columns

In [50]:
# Create a boolean mask for the rows to drop
mask = df[' Label'].isin([9, 13, 8])
# Drop the rows
df_filtered = df[~mask]

In [51]:
df_filtered.shape

(2827808, 71)

In [52]:
df.shape

(2827876, 71)

In [53]:
df_new=pd.concat([df_filtered,df_combined],axis=0)
df_new.shape

(2836512, 71)

In [55]:
# Specify the filename
filename = 'CIC_IDS_2017_merged_filtered_data.csv'

# Save the DataFrame to a CSV file
df_new.to_csv(filename, index=False)

print(f"DataFrame saved to {filename}")

DataFrame saved to CIC_IDS_2017_merged_filtered_data.csv


In [66]:
X = df.iloc[:, :-1].values  # Features (excluding the label column)
y = df.iloc[:, -1].values   # Labels

In [67]:
from sklearn.feature_selection import mutual_info_classif

In [84]:
# Indian Millipede Optimization Algorithm(IMOA) based feature selection
# Define the fitness function for feature selection based on mutual information
def fitness_function(mask):
    selected_features = np.where(mask == 1)[0]
    if len(selected_features) == 0:
        return 0  # No features selected, so fitness is zero
    
    X_selected = X[:, selected_features]
    mi = mutual_info_classif(X_selected, y, discrete_features='auto')
    return np.sum(mi)  # Maximize the sum of mutual information of selected features

# Define the constraint violation function (if any)
def constraint_violation(mask):
    return 0  # No constraints in this case

# Initialize parameters
N = 20  # Population size
d = X.shape[1]  # Dimensionality of the problem (number of features)
T = 100  # Maximum iterations
T_th = 0.5  # Temperature threshold
α = 0.1  # Seasonal activity factor
β = 0.5  # Reversal factor
γ = 0.1  # Learning rate
δ = 0.01  # Step size
ε = 0.1  # Social factor
λ = 10  # Penalty coefficient
η = 0.5  # Crossover coefficient
convergence_threshold = 1e-6  # Convergence threshold for fitness improvement
no_improvement_limit = 10  # Number of iterations to wait for improvement before stopping

# Initialize population
P = np.random.randint(0, 2, (N, d))  # Binary mask for feature selection

# Evaluate initial fitness
fitness = np.array([fitness_function(mask) for mask in P])
penalized_fitness = fitness - λ * np.array([constraint_violation(mask) for mask in P])

# Main IMOA loop
t = 0
no_improvement_count = 0
best_fitness = np.max(penalized_fitness)
best_solution = P[np.argmax(penalized_fitness)]

while t < T and no_improvement_count < no_improvement_limit:
    for i in range(N):
        # Seasonal Abundance
        α_t = α * np.sin(2 * np.pi * t / T)
        P[i] = np.clip(P[i] + α_t * np.random.uniform(-1, 1, d), 0, 1).astype(int)
        
        # Obstacle Avoidance
        if penalized_fitness[i] < np.mean(penalized_fitness):  # Assuming poor fitness as below mean fitness
            P[i] = np.clip(P[i] - β * np.random.uniform(-1, 1, d), 0, 1).astype(int)
        
        # Temperature Response
        if np.random.uniform(0, 1) > T_th:
            best_idx = np.argmax(penalized_fitness)
            P[i] = np.clip(P[i] + γ * (P[best_idx] - P[i]), 0, 1).astype(int)
        
        # Resource Utilization
        gradient = (fitness_function(P[i] + δ) - fitness_function(P[i])) / δ
        P[i] = np.clip(P[i] + δ * gradient, 0, 1).astype(int)
        
        # Group Movement
        P[i] = np.clip(P[i] + ε * (np.mean(P, axis=0) - P[i]), 0, 1).astype(int)
        
        # Defensive Behavior
        if penalized_fitness[i] < np.mean(penalized_fitness):  # Assuming poor fitness as below mean fitness
            penalized_fitness[i] -= λ * constraint_violation(P[i])
        
        # Mating Behavior
        mate_idx = np.random.randint(N)
        P[i] = np.clip(η * P[i] + (1 - η) * P[mate_idx], 0, 1).astype(int)
        
        # Predator Avoidance
        if np.std(P) < 1e-6:  # Assuming low diversity as low standard deviation
            P[i] = np.random.randint(0, 2, d)
    
    # Evaluate fitness of new positions
    fitness = np.array([fitness_function(mask) for mask in P])
    penalized_fitness = fitness - λ * np.array([constraint_violation(mask) for mask in P])
    
    # Check for convergence
    current_best_fitness = np.max(penalized_fitness)
    if current_best_fitness > best_fitness + convergence_threshold:
        best_fitness = current_best_fitness
        best_solution = P[np.argmax(penalized_fitness)]
        no_improvement_count = 0
    else:
        no_improvement_count += 1
    
    # Increment iteration counter
    t += 1

selected_features = np.where(best_solution == 1 )[0]
print(selected_features)


[ 3  4  5  6  7  9 11 13 17 18 19 23 24 26 28 29 31 32 33 35 37 38 40 42
 43 44 45 46 47 49 50 51 52 54 56 57 62 64 65 67]


In [85]:
num_entries = len( selected_features)
print("Total number of selected features",num_entries)

Total number of selected features 40


In [86]:
for feature_index in selected_features:
    print(df.columns[feature_index])

 Total Backward Packets
Total Length of Fwd Packets
 Total Length of Bwd Packets
 Fwd Packet Length Max
 Fwd Packet Length Min
 Fwd Packet Length Std
 Bwd Packet Length Min
 Bwd Packet Length Std
 Flow IAT Std
 Flow IAT Max
 Flow IAT Min
 Fwd IAT Max
 Fwd IAT Min
 Bwd IAT Mean
 Bwd IAT Max
 Bwd IAT Min
 Fwd URG Flags
 Fwd Header Length
 Bwd Header Length
 Bwd Packets/s
 Max Packet Length
 Packet Length Mean
 Packet Length Variance
 SYN Flag Count
 RST Flag Count
 PSH Flag Count
 ACK Flag Count
 URG Flag Count
 CWE Flag Count
 Down/Up Ratio
 Average Packet Size
 Avg Fwd Segment Size
 Avg Bwd Segment Size
Subflow Fwd Packets
 Subflow Bwd Packets
 Subflow Bwd Bytes
Active Mean
 Active Max
 Active Min
 Idle Std
